In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

# Чтение JSON-файла с логами
with open('botsv1.json', 'r') as file:
    data = json.load(file)

# Выведем структуру первых записей для контроля
print("Первые две записи:")
print(json.dumps(data[:2], indent=2))

# Списки для хранения данных
win_event_data = []  # Данные для WinEventLog
dns_data = []        # Данные для DNS-запросов

# Цикл для обработки и сортировки данных
skipped_entries = 0  # Счётчик пропущенных записей

for entry in data:
    if entry.get('type') == 'WinEventLog':
        try:
            # Достаточно лишь присутствия EventCode
            if 'EventCode' in entry['result']:
                win_event_data.append({
                    'EventCode': entry['result']['EventCode'],
                    'Message': entry['result'].get('Message'),  # Опциональный Message
                    'DateHour': entry['result'].get('date_hour'),  # Опциональный date_hour
                    'DateMinute': entry['result'].get('date_minute'),  # Опциональный date_minute
                    # Другие поля, если нужно...
                })
            else:
                skipped_entries += 1
                print(f"Поле 'EventCode' отсутствует в логе WinEventLog: {entry}.")
                
        except KeyError as e:
            skipped_entries += 1
            print(f"Поле '{e}' отсутствует в логе WinEventLog: {entry}.")
        
    elif entry.get('type') == 'DNS':
        try:
            # Проверяем наличие обязательного поля QueryName
            if 'QueryName' in entry['result']:
                dns_data.append({
                    'QueryName': entry['result']['QueryName'],
                    # Другие поля, если нужно...
                })
            else:
                skipped_entries += 1
                print(f"Запись DNS содержит недостающие данные: {entry}.")
                
        except KeyError as e:
            skipped_entries += 1
            print(f"Поле '{e}' отсутствует в логе DNS: {entry}.")

# Формирование датафреймов
win_event_log_df = pd.DataFrame(win_event_data)
dns_log_df = pd.DataFrame(dns_data)

# Проверка размеров датафреймов
print("Размер датафрейма WinEventLog:", len(win_event_log_df))
print("Размер датафрейма DNS:", len(dns_log_df))

# Информация о пропущенных записях
print(f"Пропущено записей: {skipped_entries}")
